In [1]:
import sys
import glob
import re
import fnmatch
import math
import os
from os import listdir
from os.path import join, isfile, basename

import itertools

import numpy as np
from numpy import float32, int32, uint8, dtype, genfromtxt

import scipy
from scipy.stats import ttest_ind

import pandas as pd

import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt

import seaborn as sns

import colorsys

In [2]:
labels = [16,64,8,32,2,4,65,66,33,67,34,17,69,70,35,71,9,18,72,36,73,74,37,75,19,76,38,77,39,78,79,20,5,40,80,10,81,82,83,84,85,86,11,22,23,24,12,3,6,49,50,25,51,13,52,26,53,27,54,55,56,28,7,14,57,58,29,59,30,60,15,61,31,62,63]
label_names_file = '/groups/saalfeld/home/bogovicj/vfb/DrosAdultBRAINdomains/refData/Original_Index.tsv'

label_names = pd.read_csv( label_names_file, delimiter='\t', header=0 )
# print label_names[ label_names['Stack id'] == 11 ]['JFRCtempate2010.mask130819' ].iloc[0]
# print label_names[ label_names['Stack id'] == 70 ]['JFRCtempate2010.mask130819' ].iloc[0]

def get_label_name( label_id ):
    return label_names[ label_names['Stack id'] == label_id ]['JFRCtempate2010.mask130819' ].iloc[0]


In [3]:
templates = ['JFRCtemplate2010', 'JFRC2013_lo', 'F-antsFlip_lo', 'F-cmtkFlip_lof', 'TeforBrain_f']
reg_methods = [ 'cmtkCow', 'cmtkCOG', 'cmtkHideo', 'antsRegOwl', 'antsRegDog', 'antsRegYang' ]

In [4]:
line=3
dist_samples_f = '/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals/label_data_line{}.csv.gz'.format( line )
dist_samples_df = pd.read_csv( dist_samples_f, header=None, names=['TEMPLATE','ALG','LINE','LABEL','DISTANCE'] )

In [ ]:
# dist_samples_df.head()

print( dist_samples_df['ALG'].unique())
print( dist_samples_df['TEMPLATE'].unique())

['cmtkCow' 'cmtkCOG' 'cmtkHideo' 'antsRegDog' 'antsRegYang' 'antsRegOwl']
['F-antsFlip_lo' 'JFRC2013_lo' 'F-cmtkFlip_lof' 'TeforBrain_f'
 'JFRCtemplate2010']


In [ ]:
ray_offset = []
ray_scale = []
tlist = []
alist = []
llist = []

for t in templates:
    t_dists = dist_samples_df[ dist_samples_df.TEMPLATE == t ]
    print( 't ', t )
    
    for a in reg_methods:
        print( 'a ', a )
        a_dists = t_dists[ t_dists.ALG == a ]
        
        for l in labels:
            l_dists = a_dists[ a_dists.LABEL == l ].DISTANCE

#             print( l )
#             print( l_dists.size )

            tlist += [t]
            alist += [a]
            llist += [l]
            
            if( l_dists.size > 10 ):
                params = scipy.stats.rayleigh.fit( l_dists )
                ray_offset += [ params[0]]
                ray_scale += [ params[1]]
            else:
                ray_offset += [float('nan')]
                ray_scale += [float('nan')]

#             print( len(ray_offset))
#             print( len(ray_scale))
#             print( ' ' )
            
        # merge all labels
        tlist += [t]
        alist += [a]
        llist += [-1]
        if( a_dists.size > 10 ):
            params = scipy.stats.rayleigh.fit( a_dists )
            ray_offset += [ params[0]]
            ray_scale += [ params[1]]
        else:
            ray_offset += [float('nan')]
            ray_scale += [float('nan')]
    


t  JFRCtemplate2010
a  cmtkCow


In [ ]:
# print( len(llist) )
# print( len(tlist) )
# print( len(alist) )
# print( len(ray_offset) )
# print( len(ray_scale) )

ray_df = pd.DataFrame( {'LABEL':llist,
                        'ALG':alist,
                        'TEMPLATE':tlist,
                        'RAY_OFFSET':ray_offset,
                        'RAY_SCALE':ray_scale })

ray_df.to_csv('/nrs/saalfeld/john/projects/flyChemStainAtlas/all_evals/stats/rayleigh_params.csv')